In [1]:
import sys
sys.path.append('../')

import pandas as pd
import cobra
import re

# Load heterologous reactions
# df = pd.read_csv('../results/C00024_C06142_metabolites_reactions.csv')
df = pd.read_csv('../results/C00084_C06142_metabolites_reactions.csv')

# Load GSMM
model = cobra.io.load_json_model('../data/original/e_coli_core.json')

# Impose anaerobic conditions
model.reactions.get_by_id('EX_o2_e').bounds = (0, 1000)

# Print number of reactions and metabolites in the model
print('Number of reactions:', len(model.reactions))
print('Number of metabolites:', len(model.metabolites))

Number of reactions: 95
Number of metabolites: 72


In [2]:
# Add new line to df using pd.concat
df = df.append({'Reaction': 'R_ex', 'Equation': 'C06142 --> C06142_e'}, ignore_index=True)
df = df.append({'Reaction': 'EX_1btol', 'Equation': 'C06142_e -->'}, ignore_index=True)

/tmp/ipykernel_86333/3245232706.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Reaction': 'R_ex', 'Equation': 'C06142 --> C06142_e'}, ignore_index=True)
/tmp/ipykernel_86333/3245232706.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Reaction': 'EX_1btol', 'Equation': 'C06142_e -->'}, ignore_index=True)


In [3]:
# Add metabolites to the model
# Extract the metabolites from the equation
def get_metabolites_to_add_to_GEM(df: pd.DataFrame) -> list:
    mapping_table=str.maketrans({'-':'','+':'', '>':'','<':''})
    metabolites=[]
    for i in df.index:
        dummy= str(df.loc[i]['Equation'])
        dummy=dummy.translate(mapping_table)
        dummy=dummy.split()
        for e in dummy:
            if len(e)>2:
                metabolites.append(e)

    metabolites = list(set(metabolites))
    return metabolites

metabolites = get_metabolites_to_add_to_GEM(df)

# Add metabolites to the model
for m in metabolites:
    if m not in model.metabolites:
        model.add_metabolites([cobra.Metabolite(m, compartment='c')])

print('Number of metabolites:', len(model.metabolites))

for i in range(len(df)):
    reaction = cobra.Reaction(df['Reaction'].iloc[i])
    model.add_reactions([reaction])
    model.reactions.get_by_id(df['Reaction'].iloc[i]).reaction = df['Equation'].iloc[i]
    # model.reactions.get_by_id(df['Reaction'].iloc[i]).bounds = (-1000, 1000)

model.metabolites.get_by_id('C06142_e').compartment='e'     
# model.reactions.get_by_id('EX_1btol').bounds = (-1000, 0)

print('Number of reactions:', len(model.reactions))

for r in df['Reaction']:
    print(model.reactions.get_by_id(r))


Number of metabolites: 79
Number of reactions: 104
R1: acald_c + coa_c + nad_c --> accoa_c + h_c + nadh_c
R2: 2.0 accoa_c --> C00332 + coa_c
R3: C01144 + nad_c <-- C00332 + h_c + nadh_c
R4: C01144 --> C00877 + h2o_c
R5: C00136 + nad_c <-- C00877 + h_c + nadh_c
R6: C01412 + coa_c + nad_c <-- C00136 + h_c + nadh_c
R7: C01412 + h_c + nadh_c --> C06142 + nad_c
R_ex: C06142 --> C06142_e
EX_1btol: C06142_e --> 


In [4]:
model.objective='EX_1btol'
solution=model.optimize()
the_yield="{:.0%}".format(solution.objective_value/abs(model.reactions.EX_glc__D_e.lower_bound))
print('The theoretical yield', the_yield)
print(solution)

The theoretical yield 100%
<Solution 10.000 at 0x7f77461b43d0>


In [5]:
from cobra.flux_analysis import flux_variability_analysis

model.objective = model.reactions.get_by_id('BIOMASS_Ecoli_core_w_GAM')
# model.objective = model.reactions.get_by_id('EX_1btol')
flux_variability_analysis(model, fraction_of_optimum=0.8)

,minimum,maximum
PFK,8.067832,13.653567
PFL,0.000000,18.243739
PGI,5.600864,9.965287
PGK,-19.549869,-16.506291
PGL,0.000000,4.364423
...,...,...
R5,-5.977891,0.000000
R6,-5.977891,0.000000
R7,0.000000,5.977891
R_ex,0.000000,5.977891
